# Einzelfunde (Ubangi)

* [Spitzbodige Flaschen am Ubangi](#Spitzbodige-Flaschen-am-Ubangi)
    * [Technologische Merkmale (fabrics)](#Technologische-Merkmale-(fabrics))
    * [Brennfarbe](#Brennfarbe)
    * [Proportionen](#Proportionen)
    * [Verzierungen](#Verzierungen)

In [1]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

## Spitzbodige Flaschen

In [8]:
sql = """SELECT
           t_Ort.ort_kurz AS SITE,
           t_Komplex.bef_nr AS FEAT,
           t_Obj.Individuum AS IND,
           t_Obj.objID,
           't_Ort'.'ort_name' || ' (Fpl. ' || 't_ort'.'Kat-Nr' || ')' AS Ort,
           't_Ort'.'ort_kurz' || ' ' || 't_Komplex'.'bef_nr' AS Komplex,
           t_Ort.ort_fluss,
           t_Obj.GefEinheit AS GE,
           t_Obj.Scherbe,
           t_Obj.Gewicht,
           t_Obj.Anzahl,
           t_Obj.Typ,
           t_Obj.Form_Gef
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Obj.Form_Gef) Like '%A1%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,SITE,FEAT,IND,objID,Ort,Komplex,ort_fluss,GE,Scherbe,Gewicht,Anzahl,Typ,Form_Gef
0,BKW,85/101,8,1099,Bokwango (Fpl. 190),BKW 85/101,Ubangi,2,G,2500,1,BOT,A1 (?)
1,LKA,85/101,a,5692,Loka (Fpl. 193),LKA 85/101,Ubangi,None,B,677,1,None,A1
2,BLU,85/101,23,5725,Bolumbu (Fpl. 194),BLU 85/101,Ubangi,None,B,497,1,None,A1
3,BLU,85/101,24,5726,Bolumbu (Fpl. 194),BLU 85/101,Ubangi,None,G,693,1,None,A1


In [7]:
pl = pd.read_csv('../plates/plates.csv')

df_merge = pd.merge(pl, df, on=['SITE', 'FEAT', 'IND'], how='inner')
df_merge

,TAF,OBJ,SITE,FEAT,NOTE,IND,objID,Ort,Komplex,ort_fluss,GE,Scherbe,Gewicht,Anzahl,Typ,Form_Gef
0,2,2,BKW,85/101,NaN,8,1099,Bokwango (Fpl. 190),BKW 85/101,Ubangi,2,G,2500,1,BOT,A1 (?)
1,4,8,BLU,85/101,NaN,23,5725,Bolumbu (Fpl. 194),BLU 85/101,Ubangi,None,B,497,1,None,A1


### Technologische Merkmale (*fabrics*)

In [3]:
sql = """SELECT
           t_Obj.objID,
           't_Ort'.'ort_name' || ' (Fpl. ' || 't_ort'.'Kat-Nr' || ')' AS Ort,
           t_Obj.Anzahl,
           t_Obj.MagerungGröße,
           t_Obj.MagerungArt,
           t_Obj.MagerungDichte,
           t_Obj.Fabric
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Obj.Form_Gef) Like '%G11c%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)

df_pivot = df.pivot_table(values = 'Anzahl', index = 'Fabric', aggfunc = sum)
df_pivot = pd.DataFrame(df_pivot)
df_pivot = df_pivot.sort_index(by=['Anzahl'], ascending=[False])

df_pivot['Prozent'] = df_pivot['Anzahl']/df_pivot['Anzahl'].sum()*100
display(df_pivot.head())

,Anzahl,Prozent
Fabric,,
1d,3,75
1c,1,25


In [4]:
a = df.loc[df['MagerungDichte'] != ''] # nur Einträgen, bei denen diese Felder ausgefüllt sind
df_pivot = a.pivot_table(values = 'Anzahl', index = 'MagerungDichte', aggfunc = sum)
custom_dict = {'sehr wenig':0,'wenig':1,'mittel':3,'viel':4, 'sehr viel':5}
df_pivot = pd.DataFrame(df_pivot, index=sorted(custom_dict, key=custom_dict.get))
df_pivot['Prozent'] = df_pivot['Anzahl']/df_pivot['Anzahl'].sum()*100
df_pivot

,Anzahl,Prozent
sehr wenig,NaN,NaN
wenig,1,100
mittel,NaN,NaN
viel,NaN,NaN
sehr viel,NaN,NaN


In [5]:
df_pivot = a.pivot_table(values = 'Anzahl', index = 'MagerungGröße', aggfunc = sum)
custom_dict = {'VF':0,'F':1,'M':3,'C':4, 'VC':5}
df_pivot = pd.DataFrame(df_pivot, index=sorted(custom_dict, key=custom_dict.get))
df_pivot['Prozent'] = df_pivot['Anzahl']/df_pivot['Anzahl'].sum()*100
df_pivot

,Anzahl,Prozent
VF,4,100
F,NaN,NaN
M,NaN,NaN
C,NaN,NaN
VC,NaN,NaN


In [6]:
df_pivot = a.pivot_table(values = 'Anzahl', index = 'MagerungArt', aggfunc = sum)
df_pivot = pd.DataFrame(df_pivot)
df_pivot = df_pivot.sort_index(by=['Anzahl'], ascending=[False])
df_pivot['Prozent'] = df_pivot['Anzahl']/df_pivot['Anzahl'].sum()*100
df_pivot

,Anzahl,Prozent
MagerungArt,,
S,4,100


### Brennfarbe

In [7]:
sql = """SELECT
           t_Obj.objID,
           [t_Obj].[Außenfarbe] || '/' || [t_Obj].[Prof_aussen] || '/' || [t_Obj].[Prof_Kern] || '/' || [t_Obj].[Prof_innen] || '/' || [t_Obj].[Innenfarbe] AS Farbe
       FROM (t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Obj.Form_Gef) Like '%G11c%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)

a = pd.Series({'Weiß':len(df[df['Farbe'].str.contains("W")].index),
               'Rot':len(df[df['Farbe'].str.contains("R")].index),
               'Sonstig':len(df[~df['Farbe'].str.contains("W|R")].index)})
a = pd.DataFrame(a)
a['Prozent'] = a[0]/a[0].sum()*100
a

,0,Prozent
Rot,0,0
Sonstig,1,25
Weiß,3,75


### Proportionen

In [47]:
sql = """SELECT
           [t_Ort].[ort_kurz] || ' ' || [t_Komplex].[bef_nr] || ':' || [t_Obj].[Individuum] AS Ind,
           t_Obj.muendungsD,
           t_Obj.muendungsH,
           t_Obj.minD,
           t_Obj.minD_H,
           t_Obj.maxD,
           t_Obj.maxD_H,
           t_Obj.bodenD,
           t_obj.Form_Gef
       FROM (t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Obj.Form_Gef) Like '%A1%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)

df['muendungsD'] = pd.to_numeric(df['muendungsD'])
#df['muendungsH'] = df['muendungsH'].convert_objects(convert_numeric=True)
#df['minD'] = df['minD'].convert_objects(convert_numeric=True)
#df['minD_H'] = df['minD_H'].convert_objects(convert_numeric=True)
#df['maxD'] = df['maxD'].convert_objects(convert_numeric=True)
#df['maxD_H'] = df['maxD_H'].convert_objects(convert_numeric=True)
#df['bodenD'] = df['bodenD'].convert_objects(convert_numeric=True)

# Verhältnis aus Maximaler Durchmesser / Mündungshöhe
#df['maxD-muendH'] = df['maxD'] / df['muendungsH']

display(df.head())

,Ind,muendungsD,muendungsH,minD,minD_H,maxD,maxD_H,bodenD,Form_Gef
0,BKW 85/101:8,10.5,None,95.0,NaN,33,NaN,NaN,A1 (?)
1,BLU 85/101:23,NaN,None,NaN,NaN,12,17.0,0.0,A1
2,BLU 85/101:24,NaN,None,5.0,25.0,10,17.0,0.0,A1
3,LKA 85/101:a,NaN,None,NaN,NaN,16,20.0,0.0,A1


### Verzierungen

In [9]:
sql = """SELECT
           t_Obj.objID,
           t_K_Pos.posID,
           t_K_Verz.verzID,
           [t_K_Pos].[posID] || ' ' || [posName] AS pos,
           t_K_Verz.verzName,
           t_K_Verz.verzBeschr,
           t_Obj.Typ,
           t_Obj.Form_Gef
       FROM t_Obj INNER JOIN "t_ObjPosVerz" ON t_Obj.objID = "t_ObjPosVerz".objID
           INNER JOIN t_K_Pos ON "t_ObjPosVerz".posID = t_K_Pos.posID
           INNER JOIN t_K_Verz ON "t_ObjPosVerz".verzID = t_K_Verz.verzID
       WHERE (((t_K_Pos.posID)<>1)
           AND ((t_K_Verz.verzName) Like 'V%')
           AND ((t_Obj.Form_Gef) Like '%G11c%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
# display(df.head())

In [10]:
df_pivot = df.pivot_table(values = 'objID', index = 'pos', columns = 'verzName', aggfunc = len)
df_pivot = df_pivot.fillna(0)
# Sortierung für graphische Darstellung umdrehen
df_pivot = df_pivot.sort_index(ascending=True)

In [11]:
#fig = plt.figure()
#fig.set_size_inches(10, 7.5)
plt.imshow(df_pivot, interpolation='nearest', cmap='Greys', alpha = .5)
plt.ylabel('Gefäßposition')
plt.xlabel('Verzierungselemente')
plt.yticks(range(len(df_pivot.index)), df_pivot.index)
plt.xticks(range(len(df_pivot.columns)), df_pivot.columns, rotation='vertical')
# plt.colorbar(orientation='horizontal')
for (i, j), z in np.ndenumerate(df_pivot):
    plt.text(j, i, '{:0.0f}'.format(z), ha='center', va='center')
plt.tight_layout()
plt.show()

## Verbreitung

Nordwestliches Kongobecken:

In [35]:
sql = """SELECT
           t_Obj.objID,
           't_Ort'.'Kat-Nr',
           't_Ort'.'ort_name' || ' (Fpl. ' || 't_ort'.'Kat-Nr' || ')' AS Ort,
           t_Obj.Typ,
           t_Ort.y_lat AS Y, 
           t_Ort.x_long AS X
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Obj.Form_Gef) Like '%A1%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df_pivot = df.pivot_table(values = 'objID', index = ['Ort', 'Kat-Nr', 'X', 'Y'], aggfunc = len)
df_pivot = df_pivot.reset_index()
df_pivot

,Ort,Kat-Nr,X,Y,objID
0,Bokwango (Fpl. 190),190,17.780500,0.102769,1
1,Bolumbu (Fpl. 194),194,17.882294,0.677829,2
2,Loka (Fpl. 193),193,17.945614,0.320338,1


Inneres Kongobecken:

In [36]:
sql = """SELECT
           't_Ort'.'ort_name' || ' (Fpl. ' || 't_ort'.'Kat-Nr' || ')' AS Ort,
           't_Ort'.'Kat-Nr',
           t_Ort.y_lat AS Y, 
           t_Ort.x_long AS X
       FROM t_Ort
       WHERE ((t_Ort.ort_name) Like '%Bolongo%')""".replace('\n',' ')

icb = pd.read_sql(sql, con)
icb['objID'] = ''
icb

,Ort,Kat-Nr,Y,X,objID
0,Bolongo (Fpl. 139),139,0.773259,18.514476,


In [37]:
df_pivot = df_pivot.append(icb)

Vermeintliches Herstellungszentrum in Bomongo:

In [38]:
df_pivot.loc[len(df)]=['', 'Bomongo','18.35054', '1.37987', 'X'] 

In [39]:
df_pivot.to_csv('../data/processed/3-1-1-12_spitzbodigeFlaschen_Verbreitung.csv', index=False)
df_pivot

,Kat-Nr,Ort,X,Y,objID
0,190,Bokwango (Fpl. 190),17.7805,0.102769,1
1,194,Bolumbu (Fpl. 194),17.8823,0.677829,2
2,193,Loka (Fpl. 193),17.9456,0.320338,1
0,139,Bolongo (Fpl. 139),18.5145,0.773259,
4,,Bomongo,18.35054,1.37987,X
